<a href="https://colab.research.google.com/github/douglasmmachado/RL/blob/main/deep_q/Deep_Q_Learning_for_Lunar_Landing_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (836 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../swig4

### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [81]:
class Network(nn.Module):

  def __init__(self,
               state_size:int = 8,
               action_size:int = 4,
               seed:int = 42,
               fc1_units:int = 256,
               fc2_units:int = 256
               ) -> None:

    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, fc1_units)
    self.fc2 = nn.Linear(fc1_units, fc2_units)
    self.fc3 = nn.Linear(fc2_units, action_size)
    self.relu = nn.ReLU()

  def forward(self, state):
    x = self.relu(self.fc1(state))
    x = self.relu(self.fc2(x))
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4 # Alpha
minibatch_size = 100 # Good size for deep q learning algorithm
discount_factor = 0.99 # Gamma
exploration_rate = 1.0 # Epsilon
replay_buffer_size = int(1e5) # To stabilize training
interpolation_parameter = 1e-3 #

### Implementing Experience Replay

In [42]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("cpu")) # To use GPU if it is available
    self.capacity = capacity # Maximum size of memory buffer
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k=batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [91]:
class Agent():

  def __init__(self,
               state_size:int,
               action_size:int
               ):

    self.device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("cpu")) # To use GPU if it is available
    self.state_size = state_size
    self.action_size = action_size

    self.qnetwork_local = Network(state_size, action_size, fc1_units=64, fc2_units=64).to(self.device)
    self.qnetwork_target = Network(state_size, action_size, fc1_units=64, fc2_units=64).to(self.device)
    self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor)

  def act(self, state, eps:float=0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.qnetwork_local.eval()
    with torch.no_grad():
      action_values = self.qnetwork_local(state)
    self.qnetwork_local.train()
    if random.random() > eps:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
    Q_targets = rewards + (discount_factor * Q_targets_next * (1 - dones))
    Q_expected = self.qnetwork_local(states).gather(1, actions)
    loss = F.mse_loss(input=Q_expected, target=Q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.qnetwork_local, self.qnetwork_target, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

  def save_model(self, path):
    torch.save(self.qnetwork_local.state_dict(), path)

### Initializing the DQN agent

In [48]:
agent = Agent(state_size=state_size,action_size=number_actions)

### Training the DQN agent

In [49]:
number_episodes = 2000 # Max number of episodes to train
max_t = 1000 # Max number of time steps per episode
epsilon_starting_value = 1.0
epsilon_final_value = 0.01
epsilon_decay_rate = 0.995
scores = []
scores_window = deque(maxlen=100)
eps = epsilon_starting_value

for episode in range(1, number_episodes+1):
  state, _ = env.reset()
  score = 0
  for t in range(0, max_t):
    action = agent.act(state, eps)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_window.append(score)
  scores.append(score)
  eps = max(epsilon_final_value, epsilon_decay_rate*eps)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_window)), end="")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_window)))
  if np.mean(scores_window)>=200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode-100, np.mean(scores_window)))
    agent.save_model('lunar_lander_dqn.pth')
    break

Episode 100	Average Score: -170.01
Episode 200	Average Score: -120.90
Episode 300	Average Score: -42.35
Episode 400	Average Score: 37.28
Episode 500	Average Score: 139.52
Episode 600	Average Score: 198.58
Episode 603	Average Score: 200.31
Environment solved in 503 episodes!	Average Score: 200.31


## Part 3 - Visualizing the results

In [50]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

# A new environment

In [92]:
import gymnasium as gym

# The Car racing environment was upgraded to v3
env = gym.make('MountainCar-v0')

# Get the shape of the state space
state_shape = env.observation_space.shape

# Get the size of the state space
state_size = env.observation_space.shape[0]

# Get the number of actions (for continuous action spaces)
number_actions = env.action_space.n

print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (2,)
State size:  2
Number of actions:  3


In [93]:
learning_rate = 1e-4 # Alpha
minibatch_size = 64 # Good size for deep q learning algorithm
discount_factor = 0.99 # Gamma
exploration_rate = 1.0 # Epsilon
replay_buffer_size = int(1e5) # To stabilize training
interpolation_parameter = 1e-3 #

In [94]:
agent_driver = Agent(state_size=state_size,action_size=number_actions)

In [98]:
import numpy as np
import torch
from collections import deque

number_episodes = 15000  # Max number of episodes to train
max_t = 200  # Max number of time steps per episode
epsilon_starting_value = 1.0
epsilon_final_value = 0.01
epsilon_decay_rate = 0.9
scores = []
scores_window = deque(maxlen=100)
eps = epsilon_starting_value

# Assuming your agent has a method to preprocess the state
def preprocess_state(state):
    return state.flatten()

for episode in range(1, number_episodes+1):
    state, _ = env.reset()
    state = preprocess_state(state)  # Preprocess the state
    score = 0
    for t in range(0, max_t):
        action = agent_driver.act(state, eps)
        next_state, reward, done, _, _ = env.step(action)
        next_state = preprocess_state(next_state)  # Preprocess the next state
        agent_driver.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores_window.append(score)
    scores.append(score)
    eps = max(epsilon_final_value, epsilon_decay_rate*eps)
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_window)), end="")
    if episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f} '.format(episode, np.mean(scores_window)))
    if np.mean(scores_window) >= -110.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode-100, np.mean(scores_window)))
        agent_driver.save_model('driver.pth')
        break

Episode 100	Average Score: -150.43 
Episode 200	Average Score: -136.97 
Episode 300	Average Score: -129.29 
Episode 400	Average Score: -129.15 
Episode 500	Average Score: -138.76 
Episode 600	Average Score: -131.91 
Episode 700	Average Score: -131.24 
Episode 800	Average Score: -133.91 
Episode 900	Average Score: -133.10 
Episode 1000	Average Score: -136.62 
Episode 1100	Average Score: -139.49 
Episode 1200	Average Score: -143.09 
Episode 1300	Average Score: -143.63 
Episode 1400	Average Score: -145.70 
Episode 1500	Average Score: -136.85 
Episode 1600	Average Score: -137.08 
Episode 1700	Average Score: -142.19 
Episode 1800	Average Score: -144.74 
Episode 1900	Average Score: -143.40 
Episode 2000	Average Score: -149.64 
Episode 2100	Average Score: -155.49 
Episode 2200	Average Score: -160.07 
Episode 2300	Average Score: -154.22 
Episode 2400	Average Score: -156.13 
Episode 2500	Average Score: -146.55 
Episode 2600	Average Score: -137.15 
Episode 2700	Average Score: -136.10 
Episode 28

In [99]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent_driver.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent_driver, 'MountainCar-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()